<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/Benchmarking_RAPIDS_Accelerating_Scikitlearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

Sat Jan 11 19:06:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#This install will ask for 2 prompts
#Start with 0.11 and then subsequent prompt say Y (only for 0.11). If it hangs in solving dependency for more than 2 min than kill,
#restart with 0.10 install
#This script for 0.10 and 0.11 has some bugs so follow subsequent steps
!wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

dist_package_index = sys.path.index("/usr/local/lib/python3.6/dist-packages")
sys.path = sys.path[:dist_package_index] + ["/usr/local/lib/python3.6/site-packages"] + sys.path[dist_package_index:]
sys.path
if os.path.exists('update_pyarrow.py'): ## This file only exists if you're using RAPIDS version 0.11 or higher
  exec(open("update_pyarrow.py").read(), globals())

sys.path.append('/usr/local/lib/python3.6/site-packages/')

--2020-01-11 19:06:17--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rapidsai/notebooks-contrib/raw/master/utils/rapids-colab.sh [following]
--2020-01-11 19:06:17--  https://github.com/rapidsai/notebooks-contrib/raw/master/utils/rapids-colab.sh
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/rapids-colab.sh [following]
--2020-01-11 19:06:18--  https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/rapids-colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.co

In [0]:
#only for 0.10
!conda install -y --prefix /usr/local -c rapidsai/label/xgboost -c rapidsai -c nvidia -c conda-forge dask-cudf xgboost

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [0]:
!ls /usr/lib/libxgboost.so

ls: cannot access '/usr/lib/libxgboost.so': No such file or directory


In [0]:
!cp /usr/local/lib/libxgboost.so /usr/lib

In [0]:
import cudf

In [0]:
from __future__ import print_function
import sys,tempfile, urllib, os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name='covertype', version=4)

In [0]:
covtyp.data.shape

(581012, 54)

In [0]:
np.unique(covtyp.target)

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [0]:
!nvidia-smi

Sat Jan 11 19:14:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
cov_df = pd.DataFrame(data= np.c_[covtyp['data'], covtyp['target']],
                     columns= covtyp['feature_names'] + ['target'])

In [0]:
cov_df.memory_usage(index=True).sum()

255645408

In [0]:
cov_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,target
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [0]:
print ("Rows     : " ,cov_df.shape[0])
print ("Columns  : " ,cov_df.shape[1])

Rows     :  581012
Columns  :  55


In [0]:
cov_df.target.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [0]:
cov_df.dtypes

Elevation                             object
Aspect                                object
Slope                                 object
Horizontal_Distance_To_Hydrology      object
Vertical_Distance_To_Hydrology        object
Horizontal_Distance_To_Roadways       object
Hillshade_9am                         object
Hillshade_Noon                        object
Hillshade_3pm                         object
Horizontal_Distance_To_Fire_Points    object
Wilderness_Area1                      object
Wilderness_Area2                      object
Wilderness_Area3                      object
Wilderness_Area4                      object
Soil_Type1                            object
Soil_Type2                            object
Soil_Type3                            object
Soil_Type4                            object
Soil_Type5                            object
Soil_Type6                            object
Soil_Type7                            object
Soil_Type8                            object
Soil_Type9

In [0]:
for cols in cov_df.columns:
  cov_df[cols] = pd.to_numeric(cov_df[cols], downcast='float')

In [0]:
cov_df['target'] = cov_df['target'].astype(np.int32)

In [0]:
cov_df['target'] = cov_df['target']-1

In [0]:
cov_df_X = cov_df.copy()
cov_df_y =  cov_df_X.pop('target')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(cov_df_X, cov_df_y, train_size=0.95, test_size=0.05)

In [0]:
X_train_gdf = cudf.DataFrame.from_pandas(X_train)
X_test_gdf = cudf.DataFrame.from_pandas(X_test)
y_train_gdf = cudf.DataFrame.from_pandas(pd.DataFrame(y_train))
y_test_gdf = cudf.DataFrame.from_pandas(pd.DataFrame(y_test))

In [0]:
!nvidia-smi

Sat Jan 11 19:14:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    35W / 250W |    605MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from cuml import RandomForestClassifier as cuRF
import time

In [0]:
cu_rf_params = {
    'n_estimators': 250,
    'max_depth': 8,
    'n_streams': 1,
    'split_algo': 0, 
    'seed': 1010
}
cu_rf = cuRF(**cu_rf_params)

In [0]:
start_time = time.time()
cu_rf.fit(X_train_gdf, y_train_gdf)
print("GPU Training Time with GPU dataframe: %s seconds" % (str(time.time() - start_time)))

GPU Training Time with GPU dataframe: 3.4566354751586914 seconds


In [0]:
predvalue = cu_rf.predict(X_test_gdf)

In [0]:
predvalue[0]

1.0

In [0]:
#cu_rf.score(X_test_gdf, y_test_gdf) --> use GPU frame if using 0.11 version of rapids
cu_rf.score(X_test_gdf, y_test.to_numpy())

0.6940897107124329

In [0]:
pred_out=predvalue.copy_to_host()

In [0]:
 from sklearn.metrics import confusion_matrix

In [0]:
confusion_matrix(y_test, pred_out)

array([[ 6301,  4255,     0,     0,     0,     0,     0],
       [ 2196, 12032,     0,     0,     0,     0,     0],
       [    0,  1765,     0,     0,     0,     0,     0],
       [    0,   140,     0,     0,     0,     0,     0],
       [    0,   462,     0,     0,     0,     0,     0],
       [    0,   905,     0,     0,     0,     0,     0],
       [  154,   841,     0,     0,     0,     0,     0]])

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
clf = RandomForestClassifier(n_estimators=250, max_depth=8, random_state=1010)

In [0]:
start_time = time.time()
clf.fit(X_train,y_train)
print("scikit learn training time: %s seconds" % (str(time.time() - start_time)))

scikit learn training time: 118.68698120117188 seconds


In [0]:
predict_scikit=clf.predict(X_test)

In [0]:
predict_scikit

array([1, 1, 0, ..., 0, 2, 6], dtype=int32)

In [0]:
clf.score(X_test,y_test)

0.7144332380985164

In [0]:
confusion_matrix(y_test, predict_scikit)

array([[ 7770,  2778,     0,     0,     0,     0,     8],
       [ 2926, 11193,   108,     0,     0,     1,     0],
       [    0,   315,  1450,     0,     0,     0,     0],
       [    0,     0,   104,    36,     0,     0,     0],
       [    0,   458,     4,     0,     0,     0,     0],
       [    0,   362,   532,     1,     0,    10,     0],
       [  695,     4,     0,     0,     0,     0,   296]])